In [4]:
from fast_graphrag import GraphRAG

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 엔티티
ENTITY_TYPES = ["Telecom", "Plan", "DataService", "CallService", "MessageService", "Benefit"]

# 도메인 설명
DOMAIN = "대한민국 3대 통신사(SK텔레콤, KT, LG U+)의 요금제 정보 및 비교 분석"

# 예시 질의/질문패턴 정의
EXAMPLE_QUERIES = [
    "SK텔레콤의 5G 요금제 중 데이터 무제한인 상품 알려줘",
    "월 3만원 이하의 요금제 추천해줘",
    "LG U+와 KT 중 영상통화 무제한 요금제는 어느 회사가 더 저렴해?",
    "데이터 소진 후 속도 제한이 없는 요금제는?",
    "가족 간 데이터 공유 가능한 요금제 알려줘",
    "학생에게 추천하는 요금제 뭐가 있어?"
]

# GraphRAG 인스턴스 생성
telecom_rag = GraphRAG(
    working_dir="./telecom_plans_graph",
    domain=DOMAIN,
    example_queries="\n".join(EXAMPLE_QUERIES),
    entity_types=ENTITY_TYPES
)


In [ ]:
import pandas as pd

df = pd.read_csv("rate_plan.csv")

# 마크다운
structured_texts = []
for _, row in df.iterrows():
    text = f"""
    통신사: {row['telecom']}
    요금제명: {row['product_name']}
    설명: {row['description']}
    네트워크: {row['mobile_generation']}
    기본 가격: {row['origin_price']}
    월 할인: {row['monthly_discount_price']}
    데이터: {row['data_amount']}
    데이터 공유: {row['sharing_data_amount']}
    속도 제한: {row['speed_after_data_consumption']}
    영상통화: {row['video_call_amount']}
    음성통화: {row['voice_amount']}
    메시지: {row['message_amount']}
    추가혜택: {row['additional_benefit']}
    """
    structured_texts.append(text)

# 데이터 삽입
for text in structured_texts:
    telecom_rag.insert(text)

Building [done]: 100%|██████████| 7/7 [00:03<00:00,  1.90it/s]                   


In [16]:
def process_query(user_query):
    # 사용자 질의 처리
    response = telecom_rag.query(user_query)
    
    # 결과 출력
    print("질문:", user_query)
    print("\n답변:")
    print(response.response)
    
    return response.response


In [14]:
process_query("시니어 요금제 추천")

질문: 시니어 요금제 추천

답변:
추천할 만한 시니어 요금제는 다음과 같습니다:

1. **5G 시니어 A형**: 월 데이터 10GB 제공, 무제한 음성통화, 400분 영상통화, 기본 메시지 서비스 포함. 기본 가격은 45,000원입니다. 

2. **5G 시니어 B형**: 월 데이터 9GB 제공, 무제한 음성통화, 400분 영상통화, 기본 메시지 서비스 포함. 기본 가격은 44,000원입니다. 

이 두 요금제는 65세 이상 고객을 위한 전용 요금제로, 5G 네트워크를 사용하며 다양한 혜택이 있습니다.


'추천할 만한 시니어 요금제는 다음과 같습니다:\n\n1. **5G 시니어 A형**: 월 데이터 10GB 제공, 무제한 음성통화, 400분 영상통화, 기본 메시지 서비스 포함. 기본 가격은 45,000원입니다. \n\n2. **5G 시니어 B형**: 월 데이터 9GB 제공, 무제한 음성통화, 400분 영상통화, 기본 메시지 서비스 포함. 기본 가격은 44,000원입니다. \n\n이 두 요금제는 65세 이상 고객을 위한 전용 요금제로, 5G 네트워크를 사용하며 다양한 혜택이 있습니다.'

In [15]:
process_query("31살이 가입할 수 있는 스마트기기 할인 가능한 SKT 요금제 추천")

질문: 31살이 가입할 수 있는 스마트기기 할인 가능한 SKT 요금제 추천

답변:
추천할 수 있는 SKT 요금제는 '0 청년 109(스마트 기기)'입니다. 이 요금제는 만 19세 이상 34세 이하 개인고객이 가입할 수 있으며, 무제한 데이터와 스마트 기기 할인 혜택이 포함되어 있습니다.


"추천할 수 있는 SKT 요금제는 '0 청년 109(스마트 기기)'입니다. 이 요금제는 만 19세 이상 34세 이하 개인고객이 가입할 수 있으며, 무제한 데이터와 스마트 기기 할인 혜택이 포함되어 있습니다."

In [17]:
process_query("31살이 가입할 수 있는 가장 저렴한 LG U+ 요금제")

질문: 31살이 가입할 수 있는 가장 저렴한 LG U+ 요금제

답변:
The most affordable LG U+ plan available for a 31-year-old is the '유쓰 5G 다이렉트 30' plan, which offers 10GB of data, unlimited voice calls, and basic messaging services, priced at 30,000 won after a monthly discount.


"The most affordable LG U+ plan available for a 31-year-old is the '유쓰 5G 다이렉트 30' plan, which offers 10GB of data, unlimited voice calls, and basic messaging services, priced at 30,000 won after a monthly discount."

In [19]:
process_query("31살이 가입할 수 있는 가장 저렴한 LG U+ 요금제와 SKT 요금제를 한국어로 비교해줘")

질문: 31살이 가입할 수 있는 가장 저렴한 LG U+ 요금제와 SKT 요금제를 한국어로 비교해줘

답변:
31살이 가입할 수 있는 가장 저렴한 LG U+ 요금제는 유쓰 5G 다이렉트 46으로, 온라인 전용 요금제입니다. SKT에서 가입할 수 있는 가장 저렴한 요금제는 0 청년 43으로, 이 요금제는 월 8GB 데이터와 다양한 청년 특화 혜택을 제공합니다. 가격은 SKT의 0 청년 43이 약 43,000원이지만, LG U+의 요금제 가격은 제공되지 않아 비교할 수 없습니다.


'31살이 가입할 수 있는 가장 저렴한 LG U+ 요금제는 유쓰 5G 다이렉트 46으로, 온라인 전용 요금제입니다. SKT에서 가입할 수 있는 가장 저렴한 요금제는 0 청년 43으로, 이 요금제는 월 8GB 데이터와 다양한 청년 특화 혜택을 제공합니다. 가격은 SKT의 0 청년 43이 약 43,000원이지만, LG U+의 요금제 가격은 제공되지 않아 비교할 수 없습니다.'

In [21]:
process_query("청년 요금제 중 넷플릭스나 유튜브 가입 가능한 요금제를 한국어로 비교해줘")

질문: 청년 요금제 중 넷플릭스나 유튜브 가입 가능한 요금제를 한국어로 비교해줘

답변:
청년 요금제 중에서 넷플릭스나 유튜브 가입이 가능한 요금제는 다음과 같습니다: 

1. **0 청년 99(넷플릭스)**  
   - 설명: 무제한 데이터와 넷플릭스 멤버십을 제공하는 요금제로, 만 19세 이상 34세 이하 개인 고객이 가입 가능.  
   - 데이터: 무제한  
   - 추가 혜택: T멤버십 VIP, 태블릿/스마트워치 요금 할인, 0청년 영화할인, 0청년 로밍할인, 0청년 커피할인.  

2. **0 청년 다이렉트 62(넷플릭스)**  
   - 설명: 무제한 데이터와 넷플릭스 멤버십을 제공하는 온라인 전용 무약정 요금제로, 만 19세 이상 34세 이하 개인 고객이 가입 가능.  
   - 데이터: 무제한  
   - 추가 혜택: T멤버십 VIP, 태블릿/스마트워치 요금 할인, 0청년 영화할인, 0청년 로밍할인, 0청년 커피할인.  

3. **0 청년 109(유튜브 프리미엄)**  
   - 설명: 무제한 데이터와 유튜브 프리미엄 멤버십을 제공하는 요금제로, 만 19세 이상 34세 이하 개인 고객이 가입 가능.  
   - 데이터: 무제한  
   - 추가 혜택: T멤버십 VIP, 태블릿/스마트워치 요금 할인, 0청년 영화할인, 0청년 로밍할인, 0청년 커피할인.


'청년 요금제 중에서 넷플릭스나 유튜브 가입이 가능한 요금제는 다음과 같습니다: \n\n1. **0 청년 99(넷플릭스)**  \n   - 설명: 무제한 데이터와 넷플릭스 멤버십을 제공하는 요금제로, 만 19세 이상 34세 이하 개인 고객이 가입 가능.  \n   - 데이터: 무제한  \n   - 추가 혜택: T멤버십 VIP, 태블릿/스마트워치 요금 할인, 0청년 영화할인, 0청년 로밍할인, 0청년 커피할인.  \n\n2. **0 청년 다이렉트 62(넷플릭스)**  \n   - 설명: 무제한 데이터와 넷플릭스 멤버십을 제공하는 온라인 전용 무약정 요금제로, 만 19세 이상 34세 이하 개인 고객이 가입 가능.  \n   - 데이터: 무제한  \n   - 추가 혜택: T멤버십 VIP, 태블릿/스마트워치 요금 할인, 0청년 영화할인, 0청년 로밍할인, 0청년 커피할인.  \n\n3. **0 청년 109(유튜브 프리미엄)**  \n   - 설명: 무제한 데이터와 유튜브 프리미엄 멤버십을 제공하는 요금제로, 만 19세 이상 34세 이하 개인 고객이 가입 가능.  \n   - 데이터: 무제한  \n   - 추가 혜택: T멤버십 VIP, 태블릿/스마트워치 요금 할인, 0청년 영화할인, 0청년 로밍할인, 0청년 커피할인.'

In [26]:
process_query("LG U+ 요금제와 KT 요금제의 청년 요금제 중 데이터 무제한 요금제 비교(한국어로)")

질문: LG U+ 요금제와 KT 요금제의 청년 요금제 중 데이터 무제한 요금제 비교(한국어로)

답변:
KT의 데이터 무제한 요금제는 주로 5G Y 베이직 요금제로, 이 요금제는 무제한 데이터와 기본 음성 및 메시지 서비스가 포함되어 있습니다. 기본 가격은 80,000원이며, 월 할인은 60,000원으로, 사용자에게 300분의 영상통화를 제공합니다. 또한, 데이터 공유는 40GB까지 가능합니다.

LG U+의 데이터 무제한 청년 요금제는 유쓰 5G 데이터 플러스입니다. 이 요금제는 청년 고객을 대상으로 한 5G 요금제로, 무제한 데이터가 제공되며, 세부적인 가격정보나 추가 혜택은 명시되어 있지 않습니다. 

결론적으로, 두 요금제 모두 데이터 무제한을 제공하나, KT 요금제는 더 상세한 가격과 혜택이 공개되어 있습니다.


'KT의 데이터 무제한 요금제는 주로 5G Y 베이직 요금제로, 이 요금제는 무제한 데이터와 기본 음성 및 메시지 서비스가 포함되어 있습니다. 기본 가격은 80,000원이며, 월 할인은 60,000원으로, 사용자에게 300분의 영상통화를 제공합니다. 또한, 데이터 공유는 40GB까지 가능합니다.\n\nLG U+의 데이터 무제한 청년 요금제는 유쓰 5G 데이터 플러스입니다. 이 요금제는 청년 고객을 대상으로 한 5G 요금제로, 무제한 데이터가 제공되며, 세부적인 가격정보나 추가 혜택은 명시되어 있지 않습니다. \n\n결론적으로, 두 요금제 모두 데이터 무제한을 제공하나, KT 요금제는 더 상세한 가격과 혜택이 공개되어 있습니다.'